<a href="https://colab.research.google.com/github/HisakaKoji/sentence-transformers/blob/master/healsion_delish_combine_recipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content
!git clone https://github.com/sonoisa/sentence-transformers
!cd sentence-transformers; pip install -r requirements.txt
!wget -O sonobe-datasets-sentence-transformers-model.tar "https://www.floydhub.com/api/v1/resources/JLTtbaaK5dprnxoJtUbBbi?content=true&download=true&rename=sonobe-datasets-sentence-transformers-model-2"
!tar -xvf sonobe-datasets-sentence-transformers-model.tar

/content
Cloning into 'sentence-transformers'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 707 (delta 2), reused 6 (delta 2), pack-reused 701
Receiving objects: 100% (707/707), 195.58 KiB | 296.00 KiB/s, done.
Resolving deltas: 100% (475/475), done.
     |████████████████████████████████| 450kB 34.3MB/s 
     |████████████████████████████████| 17.1MB 202kB/s 
     |████████████████████████████████| 1.0MB 50.5MB/s 
     |████████████████████████████████| 870kB 46.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=f4fb02b42119ac71ec518a140dcac9f0c3edee4f6293c2fe0173c5f8b4192675
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses
--2020-03-24 03:56:19--  https://www.floydhub.com/api/v1/resources/JLTtbaaK5dprnxoJtUbBbi?content=true&download=true&rename=son

In [6]:
%cd sentence-transformers

[Errno 2] No such file or directory: 'sentence-transformers'
/content/sentence-transformers


In [0]:
%tensorflow_version 2.x
from sentence_transformers import SentenceTransformer
import numpy as np

model_path = "/content/training_bert_japanese"
model = SentenceTransformer(model_path, show_progress_bar=False)

In [0]:
import pandas as pd

In [0]:
df = pd.read_csv('all_delish.csv')

In [0]:
df.head

In [0]:
df['description'] = df['店舗名'] + ' ' + df['description'] # + ' ' + df['step']

In [0]:
df

In [0]:
df = df.rename(columns={'店舗名': 'name'})

In [0]:
df = df.dropna()

In [0]:
df = df[~df.duplicated(subset='url')]

In [0]:
df

In [0]:
sentences = df['description'].values.tolist()

In [0]:
sentence_vectors = model.encode(sentences)

In [178]:
sentence_vectors[0][0]

-0.28582418

In [0]:
import numpy as np

In [0]:
data1 = np.empty(768)

In [0]:
data1 = sentence_vectors[0]

In [186]:
data1

array([-2.85824180e-01, -1.88595176e-01, -1.11514521e+00, -9.15756106e-01,
       -7.11281598e-01,  3.97506595e-01, -8.18827808e-01, -7.94989586e-01,
       -4.28184241e-01, -2.50508487e-01,  5.57112038e-01, -3.06014359e-01,
       -4.67474051e-02, -8.71085823e-01, -3.18920195e-01,  9.27358866e-01,
       -1.31371871e-01,  2.78273731e-01, -2.82521784e-01,  5.70802331e-01,
        9.01000798e-02,  8.19639146e-01, -3.10594887e-01, -6.79367125e-01,
        4.55341876e-01,  2.88679093e-01, -2.01493818e-02, -8.92498314e-01,
        2.83107579e-01, -3.97180431e-02,  6.97811544e-01, -1.72891632e-01,
       -8.40662539e-01, -1.22591269e+00, -6.18111975e-02, -4.98434275e-01,
        1.27213418e+00,  1.39010906e-01,  7.32358158e-01,  5.12952149e-01,
        3.81987065e-01, -1.44726098e+00, -3.72421294e-01, -7.37581355e-03,
        4.81458575e-01,  1.00917459e+00,  5.20410314e-02,  1.00052214e+00,
       -1.66979134e-02, -7.41504431e-01,  2.96901673e-01, -5.58574736e-01,
        9.16160166e-01, -

In [0]:
data1 = np.vstack((data1, sentence_vectors[2] ) )

In [188]:
data1

array([[-0.28582418, -0.18859518, -1.1151452 , ...,  0.43629286,
        -0.65388376,  0.3527496 ],
       [-0.27007264,  0.6650033 , -0.3760703 , ..., -0.19129378,
         0.47221196, -0.3802646 ]], dtype=float32)

In [0]:
df_vector = pd.Series(sentence_vectors)

In [168]:
df_vector[0].shape

(768,)

In [0]:
sentences_name = df['name'].values.tolist()
sentences_url = df['url'].values.tolist()

In [17]:
from sklearn.cluster import KMeans

num_clusters = 8
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(sentence_vectors)
cluster_assignment = clustering_model.labels_

clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(sentences_name[sentence_id][:10])

for i, cluster in enumerate(clustered_sentences):
    print("Cluster ", i+1)
    print(cluster)
    print("")

Cluster  1
['なすと鶏肉の甘ポン酢', '鶏肉となすの甘酢みぞ', '鶏肉ともやしの甘辛ご', '鶏むねとキムチの卵と', 'チキンとたっぷり野菜', '鶏肉とれんこんのナゲ', 'ニラともやしの鶏つく', '鶏肉のねぎ味噌炒め', '鶏肉とれんこんの甘酢', '鶏むね肉と玉ねぎの甘', '鶏肉とピーマンのポン', '鶏もも肉と白菜のしょ', '鶏肉のチリソース炒め', '鶏むね肉ときのこのレ', '鶏むねと厚揚げの甘辛', 'じゃがいもといんげん', '鶏肉と豆腐の炒め煮', '鶏むね肉とキャベツの', '肉巻ききのこのマスタ', '鶏肉とごぼうのハニー', '鶏肉とにんにくの芽の', 'しめじと鶏肉のオイス', '鶏むね肉と大葉のマヨ', '鶏もも肉と大根の山椒', '鶏もも肉ともやしの蒸', '鶏もも肉とセロリのピ', '鶏むね肉と長芋のにん', '鶏むね肉のわさびマヨ', '鶏肉とさつまいものて', 'よだれ鶏の香味ソース', '鶏肉と焦がしねぎのあ', '鶏肉のプルコギ', '鶏すき風炒め', '鶏肉と玉ねぎのバター', '鶏むね肉の甘酢ごま焼', '鶏肉とわらびの煮物', 'えのきと鶏肉のバター', '鶏むね肉とキャベツの', '鶏レバニラ', '鶏肉としめじのマヨポ', '鶏むね肉のしっとりオ', '梅だれ油淋鶏', '鶏肉のごま照り焼き', '鶏むね肉のオイマヨよ', '鶏チャーシュー丼', '鶏むね肉のがつんとス', '照り焼きチキンの和風', '鶏むねレンジチャーシ', '鶏肉の和風ピカタ', 'カラフル野菜で彩るチ', '鶏むねとじゃがいもの', '鶏むね肉の明太チーズ', 'じゃがマヨ鶏チャーシ', '塩こんぶと大葉の焼き', '鶏手羽中のオイスター', '山芋いりつくね磯辺焼', '鶏もも肉の梅マヨソー', '鶏つみれのしょうが鍋', 'ニラダレのよだれ鶏', 'れんこんのはさみ梅照', '鶏肉と玉ねぎのつくね', '鶏むねとじゃがいもの', '鶏肉とネギの塩ダレ丼', '骨つき鶏もも肉の夏野', 'れんこんのドーム蒸し', '里芋と鶏肉の和風グラ', 'キャベツと鶏もものコ', 'まるごとピーマンと鶏', '鶏むね肉とにんじんの', 'オクラと鶏肉の梅煮', 'びっくりマンモス肉', '鶏肉と野菜の松風焼き', '鶏むね肉とポ

In [0]:
df_queries = pd.read_csv('cookpad_keyword.txt',header = None)
#df_queries = pd.read_csv('healsio_recipe.csv',header = None)

In [0]:
df = pd.read_csv('healsio_recipe.csv')
df['description'] = df['name'] # + df['ingredient'] + df['step']
queries =  df['description'].values.tolist()
sentences_names = df['name'].values.tolist()
sentences_urls = df['url'].values.tolist()


In [209]:
data1 = np.ndarray([])
import scipy.spatial

#queries = ['ゆきぽよ','美術館', 'グルメ', '究極の料理','スポーツ大会', '子供がはしゃげる','東広島市　料理','東広島市　おでかけ','古墳','アニメ','細菌','ビッグバン','甲賀忍者']
#queries = df_queries[0].values.tolist()
query_embeddings = model.encode(queries)

closest_n = 5

for i,(name,url,query, query_embedding) in enumerate(zip(sentences_names,sentences_urls,queries, query_embeddings)):
    distances = scipy.spatial.distance.cdist([query_embedding], sentence_vectors, metric="cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n======================\n\n")
    print("Query:", name + ' ' + url)
    print("\nTop 5 most similar sentences in corpus:")

    for j,(idx, distance) in enumerate(results[0:closest_n]):
        print(sentences_name[idx].strip(), sentences_url[idx].strip(),"(Score: %.4f)" % (distance / 2),idx)
        if j == 0 and i == 0:
          data1 = sentence_vectors[idx]
          print(idx)
        elif j == 0:
          data1 = np.vstack((data1, sentence_vectors[idx] ) )
          print(idx)


ストリーミング出力は最後の 5000 行に切り捨てられました。
パプリカとブロッコリーのペペロンソテー https://delishkitchen.tv/recipes/157728566329475475 (Score: 0.2103) 7780




Query: パンプキングラタン https://cook-healsio.jp/healsio/AX-XW600/recipes/R1517

Top 5 most similar sentences in corpus:
ほうれん草とたらのクリームグラタン https://delishkitchen.tv/recipes/142002002124079578 (Score: 0.2133) 6230
6230
ピーマンのゆずこしょうナムル https://delishkitchen.tv/recipes/204967512616468902 (Score: 0.2283) 9100
にんじんとベーコンのグラタン https://delishkitchen.tv/recipes/150780869676630505 (Score: 0.2297) 11661
れんこんのふわふわはさみ揚げ https://delishkitchen.tv/recipes/212543473880925402 (Score: 0.2326) 11432
マカロニの代わりに豆腐グラタン https://delishkitchen.tv/recipes/206188063167087639 (Score: 0.2326) 1185




Query: パーティおこわ https://cook-healsio.jp/healsio/AX-XW600/recipes/R0287

Top 5 most similar sentences in corpus:
ちくわキッシュ https://delishkitchen.tv/recipes/156288515787719059 (Score: 0.2586) 11739
11739
アボカドクリームチーズわさび醤油和え https://delishkitchen.tv/recipes/147633153777860902 (Score: 0.2655) 9243
フライパンパエリャ ht

In [0]:
df_h = pd.read_csv('healsio_recipe.csv')

In [0]:

df_h['description'] = df_h['name'] #  +  df['step']#+ df['ingredient'] 

In [131]:
df_h['description']

0       アイスボックスクッキーバター（柔らかくしたもの） 160g 砂糖 100g 卵 L1個 バニ...
1       アイスメレンゲカップ入りアイスクリーム(市販品) 2個(110mLサイズ/個) 卵白 1/2...
2       青菜のあえ物えのきだけ（半分に切ってほぐす） 60g しいたけ（2mm幅の薄切り） 4個（6...
3        青煮ふき 2本（60g） きぬさや 60g 白煮の煮汁（下記参照） 1カップ キッチンペーパー 
4       青野菜のとろみあえチンゲン菜 300ｇ アスパラガス 100ｇ そら豆（さや付き） 300～...
                              ...                        
1247    わかめもちごまだれがけ角もち 4個（210g) 塩わかめ 120g ごまだれ（しゃぶしゃぶ用...
1248    わたりがにの豆板醤炒めわたりがに 2杯 酒 少々 にんにく(みじん切り) ひとかけ しょうが...
1249    和風ハンバーグおろしぽん酢添え木綿豆腐 200ｇ 鶏ひき肉 240ｇ 塩 小さじ2/5 こし...
1250    グリルステーキ初夏の野菜添え牛肉(焼き肉用) 300g さやいんげん 50g そら豆 4本 ...
1251    フォカッチャ強力粉 360g 薄力粉 140g 砂糖 小さじ4 オリーブオイル  大さじ2 ...
Name: description, Length: 1252, dtype: object

In [0]:
df_h = df_h[~df_h.duplicated(subset='url')]

In [0]:
sentences =  df_h['description'].values.tolist()
sentences_name = df_h['name'].values.tolist()
sentences_url = df_h['url'].values.tolist()

In [0]:
sentence_vectors = model.encode(sentences)



In [0]:
#df_queries = pd.read_csv('cookpad_keyword.txt',header = None)
df_queries = pd.read_csv('kurashiru_keyword.txt',header = None)


In [0]:
df_out = pd.read_csv('output.txt')

In [215]:
import scipy.spatial

queries = ['ゆきぽよ','美術館', 'グルメ', '究極の料理','スポーツ大会', '子供がはしゃげる','東広島市　料理','東広島市　おでかけ','古墳','アニメ','細菌','ビッグバン','甲賀忍者']
queries = df_queries[0].values.tolist()
query_embeddings = model.encode(queries)

closest_n = 5
adata = pd.DataFrame()
for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], data1, metric="cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")
    i = 0
    for idx, distance in results[0:closest_n]:
        i+=1
        print(sentences_name[idx].strip(), sentences_url[idx].strip(),"(Score: %.4f)" % (distance / 2))
        temp = pd.DataFrame({'query': query,
                   'name':sentences_name[idx].strip(),'rank': i,
                   'url': sentences_url})
        adata = pd.concat([adata,temp], axis=0)





Query: 簡単

Top 5 most similar sentences in corpus:
お手軽おいなりさん https://cook-healsio.jp/healsio/AX-XW600/recipes/R0335 (Score: 0.2139)
お手軽チンジャオロースー https://cook-healsio.jp/healsio/AX-XW600/recipes/R0887 (Score: 0.2538)
簡単ケチャップライス https://cook-healsio.jp/healsio/AX-XW600/recipes/R1407 (Score: 0.2626)
するめ https://cook-healsio.jp/healsio/AX-XW600/recipes/R0410 (Score: 0.2643)
簡単食パンキッシュ https://cook-healsio.jp/healsio/AX-XW600/recipes/R1593 (Score: 0.2657)




Query: お酒に合う

Top 5 most similar sentences in corpus:
さばの香草焼き https://cook-healsio.jp/healsio/AX-XW600/recipes/R1464 (Score: 0.2098)
たいのみそ漬け https://cook-healsio.jp/healsio/AX-XW600/recipes/R0048 (Score: 0.2175)
たらのワイン蒸し https://cook-healsio.jp/healsio/AX-XW600/recipes/R1105 (Score: 0.2175)
蒸しなすの黒酢あん https://cook-healsio.jp/healsio/AX-XW600/recipes/R1340 (Score: 0.2310)
ぶりの酢煮(レンジメニュー) https://cook-healsio.jp/healsio/AX-XW600/recipes/R2188 (Score: 0.2356)




Query: お手軽

Top 5 most similar sentences in corpus:
お手軽おいなりさん https://cook-

In [0]:
adata.to_csv('healsio_out0324.csv')

In [77]:
adata.head

<bound method NDFrame.head of      query      name  rank                                                url
0       簡単  田作り(ごまめ)     1  https://cook-healsio.jp/healsio/AX-XW600/recip...
1       簡単  田作り(ごまめ)     1  https://cook-healsio.jp/healsio/AX-XW600/recip...
2       簡単  田作り(ごまめ)     1  https://cook-healsio.jp/healsio/AX-XW600/recip...
3       簡単  田作り(ごまめ)     1  https://cook-healsio.jp/healsio/AX-XW600/recip...
4       簡単  田作り(ごまめ)     1  https://cook-healsio.jp/healsio/AX-XW600/recip...
...    ...       ...   ...                                                ...
1247   ビール   ひらめの粕漬け     5  https://cook-healsio.jp/healsio/AX-XW600/recip...
1248   ビール   ひらめの粕漬け     5  https://cook-healsio.jp/healsio/AX-XW600/recip...
1249   ビール   ひらめの粕漬け     5  https://cook-healsio.jp/healsio/AX-XW600/recip...
1250   ビール   ひらめの粕漬け     5  https://cook-healsio.jp/healsio/AX-XW600/recip...
1251   ビール   ひらめの粕漬け     5  https://cook-healsio.jp/healsio/AX-XW600/recip...

[1878000 rows x 4 columns]>

In [0]:
%load_ext tensorboard
import os
logs_base_dir = "runs"
os.makedirs(logs_base_dir, exist_ok=True)

In [0]:
import torch
from torch.utils.tensorboard import SummaryWriter

import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

summary_writer = SummaryWriter()
summary_writer.add_embedding(mat=np.array(sentence_vectors), metadata=sentences_name)

In [0]:
%tensorboard --logdir {logs_base_dir}

In [0]:
!kill 1265